In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import sklearn

In [2]:
data = pd.read_csv('health_data.csv')
data.head()

,age,restbps,chol,category
0,26,109,243,0
1,27,106,156,0
2,28,107,225,0
3,27,105,277,0
4,30,96,221,0


In [3]:
X = data.to_numpy()[:, :3]
Y = data.to_numpy()[:, 3]



split = 0.8 #using 70:30 split, random each time
train_samples = np.random.choice(X.shape[0], int(split*X.shape[0]))
test_samples = np.array([i for i in range(X.shape[0]) if i not in list(train_samples)])

X_train = X[[train_samples]]
Y_train = Y[[train_samples]]

X_test = X[[test_samples]]
Y_test = Y[[test_samples]]


X_0 = X_train[Y_train == 0]
X_1 = X_train[Y_train == 1]



X_test_0 = X_test[Y_test == 0]
X_test_1 = X_test[Y_test==1]





#X_00 = data[data['category'] == 0].to_numpy()[:, :3]
#X_11 = data[data['category'] == 1].to_numpy()[:, :3]

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  # Remove the CWD from sys.path while we load stuff.
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  # This is added back by InteractiveShellApp.init_path()
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an

In [4]:
#mean function
def mean(data):
    return sum(data)/float(len(data))

In [5]:
#variance function
def std(data):
    mu  = mean(data)
    return math.sqrt(sum([(x - mu)**2 for x in data])/float(len(data) - 1))

In [6]:
#stat of class 0(Maximum Likelihood Estimate)
stat_X0 = [(mean(col),  std(col)) for col in zip(*X_0)]
stat_X0

[(31.432926829268293, 9.230736514831873),
 (113.34146341463415, 12.203894211992147),
 (233.2012195121951, 31.22154824733778)]

In [7]:
#stat of class 1(Maximum Likelihood Estimate)
stat_X1 = [(mean(col),  std(col)) for col in zip(*X_1)]
stat_X1

[(54.849137931034484, 9.25065104239509),
 (141.52155172413794, 12.237174970612381),
 (291.05172413793105, 60.09852586268063)]

In [8]:
stat = [stat_X0, stat_X1]

In [9]:
#Probability calculation
def Normal_dist(x, mean, std):
    return (1 / (math.sqrt(2 * math.pi) * std)) *math.exp(-((x-mean)**2 / (2 * std**2 )))

In [10]:
#prior calculation
P = [float(len(X_0))/ float(len(data)) ,float(len(X_1))/ float(len(data)) ]
P

[0.4685714285714286, 0.3314285714285714]

In [11]:
#calculation of class conditional density
def class_conditional_probability(data, stat):
    probabilities = np.ones(2, dtype = np.float64)
    for i in range(0,2):
        for j in range(len(stat[i])):
            mean, stdev = stat[i][j]
            #print( Normal_dist(data[i], mean, stdev))
            probabilities[i] *= Normal_dist(data[j], mean, stdev)
    return probabilities

In [12]:
#calculation of posterior
def posterior(data, stat, P):
    prob = np.zeros(2, dtype = np.float64)
    probabilities = class_conditional_probability(data, stat)
    for i in range(2):
        prob[i] = probabilities[i]*P[i]
    return prob

In [13]:
def prediction(data, stat,P, Th):
    Probabilities = posterior(data, stat, P)
    Probabilities = Probabilities/sum(Probabilities)
    if Probabilities[1] > Th:
        return 1
    else:
        return 0

In [14]:
#computing accuracy on training data.
C1 = np.zeros((2,2)) #[[]]
for i in range((X_0.shape[0])):
    y_pred = prediction(X_0[i], stat, P, 0.5)
    C1[y_pred, 0] += 1
for i in range((X_1.shape[0])):
    y_pred = prediction(X_1[i], stat, P, 0.5)
    C1[y_pred, 1] += 1

In [15]:
acc1 = (C1[0,0] + C1[1,1])/np.sum(C1)
print(acc1)

0.8660714285714286


In [16]:
#computing accuracy on test data.
C = np.zeros((2,2)) #[[]]
for i in range((X_test_0.shape[0])):
    y_pred = prediction(X_test_0[i], stat, P, 0.5)
    C[y_pred, 0] += 1
for i in range((X_test_1.shape[0])):
    y_pred = prediction(X_test_1[i], stat, P, 0.5)
    C[y_pred, 1] += 1

In [17]:
C

array([[155.,  24.],
       [ 25., 111.]])

In [18]:
acc = (C[0,0] + C[1,1])/np.sum(C)
print(acc)

0.8444444444444444
